In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

batch_size = 256
learning_rate = 0.0002
num_epoch = 10

<body>
train 인수 : True면 학습 데이터를 불러오고 False면 테스트 데이터를 불러온다.<br>
transform 인수 : 데이터에 대한 변형<br>
target_transform : 라벨에 대한 변형<br>
download : 현재 경로에 MINST 데이터가 없을 경우에 다운로드하겠다.<br>
</body>

In [6]:
mnist_train = dset.MNIST("./",train=True,transform = transforms.ToTensor(),target_transform=None,download=True)
mnist_test = dset.MNIST("./",train = False, transform = transforms.ToTensor(), target_transform = None,download=True)

train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,shuffle=True,num_workers=2,drop_last=True)
#num_worker : 데이터를 묶을 때 사용할 프로세스의 갯수
#drop_last : 묶고 남는 데이터는 버릴지 여부

test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,shuffle=False,num_workers=2,drop_last=True)




In [7]:
for j,[image,label] in enumerate(train_loader):
    print(label)
    if j == 0:
        break

tensor([1, 4, 2, 2, 6, 1, 1, 1, 2, 3, 8, 4, 0, 0, 3, 0, 1, 7, 5, 9, 1, 0, 0, 8,
        1, 7, 7, 7, 3, 2, 0, 2, 6, 3, 5, 4, 9, 2, 7, 0, 7, 7, 7, 8, 4, 3, 4, 6,
        6, 7, 7, 7, 3, 7, 0, 4, 2, 9, 3, 6, 4, 9, 0, 6, 5, 4, 4, 5, 8, 5, 2, 0,
        3, 5, 9, 3, 9, 3, 3, 9, 5, 0, 7, 9, 8, 8, 6, 7, 4, 4, 6, 8, 6, 8, 3, 0,
        2, 6, 0, 9, 6, 8, 0, 7, 8, 1, 8, 6, 0, 8, 7, 1, 7, 9, 2, 4, 9, 8, 9, 2,
        7, 8, 9, 8, 3, 6, 8, 4, 0, 7, 5, 0, 4, 2, 7, 4, 0, 9, 4, 3, 2, 1, 0, 9,
        7, 4, 9, 7, 7, 3, 7, 5, 4, 3, 8, 0, 0, 9, 0, 3, 5, 0, 7, 9, 1, 9, 8, 4,
        3, 7, 3, 1, 5, 0, 7, 9, 7, 5, 8, 8, 2, 1, 0, 9, 4, 2, 7, 7, 0, 2, 6, 3,
        3, 1, 6, 4, 3, 9, 8, 7, 7, 3, 9, 5, 2, 9, 1, 7, 9, 1, 6, 6, 3, 2, 3, 1,
        7, 0, 2, 4, 7, 4, 2, 9, 7, 7, 9, 1, 3, 8, 5, 6, 9, 0, 7, 0, 8, 6, 9, 1,
        0, 9, 9, 7, 9, 0, 4, 3, 1, 1, 9, 6, 8, 1, 5, 3])


<h2>CNN모델</h2>

In [8]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__() #CNN의 부모 클래스인 nn.Module을 초기화하는 역할
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),#5는 임의로 그냥 설정하였음.
            nn.ReLU(),
            nn.Conv2d(16,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),#kernel_size: 풀링 연산 할 때 한번에 훑는 영역의 크기, #stride, #padding
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),#output size: [batch_size,64,6,6]
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10)#output size : [batch_size,10]
        )
        
    def forward(self,x):#연산을 순차적으로 실행하여 결괏값만 리턴하도록 정의
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)
        return out

<h2>모델 초기화와 손실함수 지정과 optimizer지정</h2>

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device) #모델을 올려줌
loss_func = nn.CrossEntropyLoss()#classification에서는 crossentorpy loss function을 사용하는 게 좋음
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)






In [ ]:
loss_arr = []
for i in range(num_epoch):
   
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()
        
        if j%1000==0:
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())#텐서에 대한 히스토리 추적을 중지하려면 .detach()를 호출하여 현재의 계산 기록으로부터 분리시키고 이후에 일어나는 계산들은 추적되지 않게 할 수 있음.
            
        

tensor(2.3037, grad_fn=<NllLossBackward>)
tensor(0.3051, grad_fn=<NllLossBackward>)
tensor(0.1057, grad_fn=<NllLossBackward>)
tensor(0.0996, grad_fn=<NllLossBackward>)
tensor(0.1163, grad_fn=<NllLossBackward>)


<h2>모델을 테스트 데이터에 대해 검증</h2>

In [ ]:
#torch.max() : https://wingnim.tistory.com/34
#torch.max 함수는 주어진 텐서 배열의 최대 값이 들어있는 index를 리턴하는 함수입니다.
#Y_pred = [ [0.3,0.2,0.9,0.1] ] 의 경우 torch.max(Y_pred.data , 1 ) 의 결과는 0.9의 인덱스인 2가 됩니다.

In [ ]:
#초기화 해줌.
correct = 0
total = 0

with torch.no_grad():
    for image,label in test_loader:
        x = image.to(device)
        y_ = label.to(device)
        
        output = model.forward(x)
        _,output_index = torch.max(output,1)
        
        total += label.size(0)#batch
        correct += (ouput_idex==y_).sum().float()
        
    print("Accuracy of Test Data: {}".format(100*correct/total))